In [1]:
import geopandas
import leafmap

C:\Users\khale\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
con = leafmap.connect_postgis(database='GIS', host='localhost', user='postgres', password='postgres')
sqlQ = "SELECT * FROM public.utility"
gdf = leafmap.read_postgis(sqlQ, con, geom_col='geometry')

In [ ]:
gdf.head(50)

In [ ]:
facilities = geopandas.read_file("../../Data/Facilities/DemoCairo.gpkg", layer='facilities').to_crs("EPSG:3857") #Point
admin_poly = geopandas.read_file("../../Data/Facilities/DemoCairo.gpkg", layer='NewCairoPolyDemo').to_crs("EPSG:3857") #Polygon
facilities.head(50)



In [ ]:
fac_And_Admin = geopandas.sjoin(gdf,admin_poly[['adm3_ar', 'geometry']], how='left')
fac_And_Admin.rename_geometry('fac_geo', inplace=True)
fac_And_Admin_for_water = fac_And_Admin.loc[fac_And_Admin['sub_type']=='Water']
fac_And_Admin_for_water['prob_lat'] = fac_And_Admin_for_water['fac_geo'].x
fac_And_Admin_for_water['prob_lon'] = fac_And_Admin_for_water['fac_geo'].y
geometry = geopandas.points_from_xy(fac_And_Admin_for_water.prob_lon, fac_And_Admin_for_water.prob_lat)
geo_df_list = [[point.xy[0][0], point.xy[1][0]] for point in fac_And_Admin_for_water.fac_geo]
print(geo_df_list)
lastprob=geo_df_list[-1]
print(lastprob)

fac_And_Admin_for_water.head(50)


In [48]:
facilities = geopandas.read_file("../../Data/Facilities/DemoCairo.gpkg", layer='facilities').to_crs("EPSG:3857") #Point
fac_And_Admin = geopandas.sjoin(facilities,admin_poly[['adm3_ar', 'geometry']], how='left')
fac_And_Admin.rename_geometry('fac_geo', inplace=True)
fac_And_Admin_for_fire = fac_And_Admin.loc[fac_And_Admin['type']=='Utility']
toto = fac_And_Admin_for_fire.to_crs("EPSG:4326")
toto['fac_lat'] = toto['fac_geo'].y
toto['fac_lon'] = toto['fac_geo'].x
geometry = geopandas.points_from_xy(toto.fac_lon, toto.fac_lat)
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in toto.fac_geo]
geo_df_list.insert(0,lastprob)
print(geo_df_list)

[[30.132701, 31.627722], [29.9754651068569, 31.415951651482498], [29.974992487599934, 31.411211357862946], [29.98870245936914, 31.39198000403305], [30.012971451534217, 31.440178060826767], [30.012767650474146, 31.43999807986315], [30.015896107811418, 31.436299261670715], [29.983306205489907, 31.41935905404484], [30.006957517114657, 31.47180021287592], [30.010121340724876, 31.480682777668736], [30.026983688846016, 31.4705935248286], [30.054983269175157, 31.490931261198263], [30.06787323384948, 31.46364871443572], [30.079609575270663, 31.452987684242245], [30.079236840732417, 31.4527823305374], [30.055353096468732, 31.449456625579632], [30.05114359025932, 31.45235998298508], [30.050947662994286, 31.45247567494661], [30.049706781314544, 31.451152762516998], [30.064564151072574, 31.447893429979725], [30.053695111406476, 31.510322601546257], [30.16182861893628, 31.563635732646933], [30.158876936846607, 31.604927972614796], [30.158581216376703, 31.641089345701285], [30.138406944555452, 31.67

In [13]:
from xml.etree.ElementTree import tostring
import requests
import json

In [50]:
z=[]
for i in range(len(geo_df_list)-1):
    z.append(i+1)
print(z)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


In [51]:
body = {"locations":geo_df_list,"destinations":[0],"metrics":["duration","distance"],"sources":z,"units":"m"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.text)

{"durations":[[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null]],"distances":[[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null],[null]],"destinations":[null],"sources":[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],"metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"matrix","timestamp":1647816732384,"query":{"locations":[[30.132701,31.627722],[29.9754651068569,31.415951651482498],[29.974992487599934,31.411211357862946],[29.98870245936914,31.39198000403305],[30.012971451534217,31.440178060826767],[30.012767650474146,31.43999807986315],[30.015896107811418,31.436299261670715],[29

In [ ]:
print(call.status_code, call.reason)
print(call.text)
res = json.loads(call.text)
x=res['sources']
z=[]
for p in x:
    y=p['snapped_distance']
    z.append(y)
print(z)
z.sort()
m=z[0]

for i in x:
    if i['snapped_distance']==m:
        k=i['location']

print(res)
print('----------------------------')
print('closest facility:')        
print(k)

h=[31.625347137451172,30.087662159514313]
q=k[0]
w=k[1]

r=h[0]
t=h[1]
print(q)

In [34]:
fires = geopandas.read_file("../../Data/Facilities/DemoCairo.gpkg", layer='fire').to_crs("EPSG:3857") #Point
facilities = geopandas.read_file("../../Data/Facilities/DemoCairo.gpkg", layer='facilities').to_crs("EPSG:3857") #Point
admin_poly = geopandas.read_file("../../Data/Facilities/DemoCairo.gpkg", layer='NewCairoPolyDemo').to_crs("EPSG:3857") #Polygon
fires.head(50)
facilities.head(50)

,id,name,type,geometry
0,1.0,Traffic Department El-Kattameya,Road,POINT (3494299.742 3500173.815)
1,2.0,El-Kattameya Traffic,Road,POINT (3493989.186 3500371.296)
2,NaN,Third Settlement Municipal,Utility,POINT (3497207.741 3500396.501)
3,NaN,Third Settlement Social Housing,Utility,POINT (3496680.054 3500335.766)
4,NaN,Government Office,Road,POINT (3497319.960 3500555.428)
5,NaN,Sewage treatment plant,Utility,POINT (3494539.229 3502097.733)
6,NaN,North Cairo Electricity Distribution- New Cairo,Utility,POINT (3499904.612 3505217.312)
7,NaN,Fifth Settlement Electriciy,Utility,POINT (3499884.577 3505191.112)
8,NaN,Telecom Egypt,Utility,POINT (3499472.826 3505593.304)
9,NaN,New Cairo 5th Police Department,None,POINT (3498222.184 3503106.233)


In [42]:
fac_And_Admin = geopandas.sjoin(facilities,admin_poly[['adm3_ar', 'geometry']], how='left')
fac_And_Admin.rename_geometry('fac_geo', inplace=True)
fac_And_Admin_for_fire = fac_And_Admin.loc[fac_And_Admin['type']=='Fire']
fac_And_Admin_for_fire = fac_And_Admin.loc[fac_And_Admin['adm3_ar']=='Industrial Region East']
toto = fac_And_Admin_for_fire.to_crs("EPSG:4326")
toto['fac_lat'] = toto['fac_geo'].y
toto['fac_lon'] = toto['fac_geo'].x
geometry = geopandas.points_from_xy(toto.fac_lon, toto.fac_lat)
geo_df_list = [[point.xy[0][0], point.xy[1][0]] for point in toto.fac_geo]
print(geo_df_list)

facdest=geo_df_list[0]



[[31.725559186069926, 30.146057856944388]]


In [ ]:
body = {"locations":[[31.625347137451172,30.087662159514313],facdest,[31.125184898206078,30.545245933532715]],"destinations":[0],"metrics":["duration","distance"],"sources":[1,2],"units":"m"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)



In [ ]:
print(call.status_code, call.reason)
print(call.text)
res = json.loads(call.text)
x=res['sources']
z=[]
for p in x:
    y=p['snapped_distance']
    z.append(y)
print(z)
z.sort()
m=z[0]

for i in x:
    if i['snapped_distance']==m:
        k=i['location']

print(res)
print('----------------------------')
print('closest facility:')        
print(k)

h=[31.625347137451172,30.087662159514313]
q=k[0]
w=k[1]

r=h[0]
t=h[1]
print(q)

In [ ]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}
call = requests.get('https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9&start={},{}&end={},{}'.format(q,w,r,t), headers=headers)

print(call.status_code, call.reason)
print(call.text)